#  Pytorchのnnモジュール

## nn.Linear

In [4]:
from torch import nn
import torch
import numpy 
import matplotlib.pyplot as plt



In [5]:
linear = nn.Linear(64, 30)

In [7]:
list(linear.parameters())

[Parameter containing:
 tensor([[ 0.0261, -0.1195, -0.1105,  ...,  0.0393,  0.0388,  0.1087],
         [-0.1116, -0.0047,  0.0813,  ...,  0.0465,  0.0301,  0.1230],
         [ 0.0816, -0.0943,  0.0891,  ..., -0.1058, -0.0044, -0.0974],
         ...,
         [ 0.0704, -0.0698, -0.0014,  ..., -0.0789,  0.0134,  0.0802],
         [-0.0879,  0.0063, -0.0169,  ...,  0.0439,  0.0524,  0.0996],
         [ 0.0546,  0.0925, -0.0670,  ..., -0.0560, -0.0242,  0.1130]],
        requires_grad=True),
 Parameter containing:
 tensor([-0.0487, -0.1111,  0.0518,  0.1091,  0.1020,  0.0921, -0.0608, -0.1208,
         -0.0321,  0.0632, -0.0027, -0.0199, -0.1149,  0.0162, -0.0608, -0.0617,
         -0.0802, -0.0170,  0.1164,  0.0040, -0.0678,  0.0126, -0.0086, -0.0334,
         -0.0248, -0.1250,  0.1227, -0.0809,  0.1129, -0.0232],
        requires_grad=True)]

In [11]:
linear.weight.shape # W = 出力 x 入力 (ex : 30 x 64)

torch.Size([30, 64])

In [12]:
linear.bias.shape # b = 1 x 出力

torch.Size([30])

In [13]:
x = torch.randn((5, 64))
Z = linear(x)

上記のcodeはZ = X @ W.T + bを計算している  
Z = (5,64) @ (出力30, 入力64).T + b(1, 出力30)

In [15]:
Z.shape

torch.Size([5, 30])